In [103]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
from crowdnalysis.cmdstan import StanMultinomialOptimizeConsensus, \
    StanMultinomialEtaOptimizeConsensus, \
    StanDSOptimizeConsensus, \
    StanDSEtaOptimizeConsensus
from crowdnalysis.dawid_skene import DawidSkene
from crowdnalysis.majority_voting import MajorityVoting
import numpy as np
import json

In [105]:
mv = MajorityVoting()
smoc = StanMultinomialOptimizeConsensus()
smetaoc = StanMultinomialEtaOptimizeConsensus()
sdsoc = StanDSOptimizeConsensus()
sdsetaoc = StanDSEtaOptimizeConsensus()
ds = DawidSkene()

In [106]:
t = 10000
num_annotations_per_task = 20
real_tau = np.array([0.1, 0.3, 0.6])
k = len(real_tau)
real_pi = np.array([[0.5, 0.3, 0.2], [0.2, 0.6, 0.2], [0.25, 0.3, 0.45]]) 
print(real_pi)
t_C, m = smoc.sample(t, num_annotations_per_task, parameters = {'tau':real_tau, 'pi':real_pi})
print(t_C)

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.sample_tasks
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.sample_annotations
INFO:cmdstanpy:start chain 1


[[0.5  0.3  0.2 ]
 [0.2  0.6  0.2 ]
 [0.25 0.3  0.45]]


INFO:cmdstanpy:finish chain 1


[2 2 2 ... 2 2 2]


In [107]:
m

array([[   0,    0,    0],
       [   0,    0,    2],
       [   0,    0,    0],
       ...,
       [9999,    0,    1],
       [9999,    0,    0],
       [9999,    0,    2]])

In [108]:
t_A = m[:,0]+1
w_A = m[:,1]
ann = m[:,2]+1
w = 1
#t = np.unique(t_A).shape[0]
a = m.shape[0]
json_data = {'w': w,
             't': t,
             'a': a,
             'k': k,
             't_A': t_A.tolist(),
             'w_A': w_A.tolist(),
             'ann': ann.tolist()}

In [109]:
print("Max w_A:",np.max(w_A))
with open('multinomial.json', 'w') as outfile:
    json.dump(json_data, outfile, indent=2)

Max w_A: 0


In [110]:
def print_vars(d,v):
    for name in v:
        if name in d:
            print(name,"=",d[name])

In [111]:
def err_rate(t_C, t_C_m):
    max_t_C_m = np.argmax(t_C_m,axis=1)
    #print(max_t_C_m)
    return 1-(np.sum(t_C==max_t_C_m)/len(t_C))
def log_score(t_C, t_C_m):
    f = t_C_m.flatten()
    start = k*np.arange(t)
    return np.sum(np.log(f[start+t_C]))

In [112]:
print("Real parameters")
print("tau=",real_tau)
print("pi=",real_pi)
for model in [mv, smoc, ds, smetaoc, sdsoc, sdsetaoc]:
    print("*********")
    print(model.name)
    t_C_m, d = model.m_fit_and_compute_consensus(m, t, k, w)
    print("error_rate:", err_rate(t_C, t_C_m))
    print("log_score:", log_score(t_C, t_C_m))
    #print(d.keys())
    print_vars(d,["p","_pi","tau","pi"])
    #if "p" in d:
    #    print (d['p'],d['pi_'])
    #else:
    #    print(d['tau'],d['pi'])

Real parameters
tau= [0.1 0.3 0.6]
pi= [[0.5  0.3  0.2 ]
 [0.2  0.6  0.2 ]
 [0.25 0.3  0.45]]
*********
MajorityVoting


/home/cerquide/prj/crowdnalysis/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
/home/cerquide/prj/crowdnalysis/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until
INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.fit_and_consensus


error_rate: 0.22399999999999998
log_score: -inf
*********
StanMultinomialOptimize


INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


error_rate: 0.9486
log_score: -52147.86995723582
tau = [0.608736  0.0968126 0.294451 ]
pi = [[0.251518 0.299346 0.449137]
 [0.504781 0.300907 0.194312]
 [0.202022 0.600665 0.197314]]
*********
DawidSkene


...

error_rate: 0.10109999999999997
log_score: -2683.3130557378936
tau = [0.0965617 0.294497  0.608942 ]
pi = [[0.5054   0.300133 0.194467]
 [0.202153 0.60078  0.197067]
 [0.251436 0.299423 0.449141]]
*********
StanDSOptimize


INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/DSEta.fit_and_consensus


error_rate: 0.9486
log_score: -52147.86995723582
tau = [0.608736  0.0968126 0.294451 ]
pi = [[[0.251518 0.299346 0.449137]
  [0.504781 0.300907 0.194312]
  [0.202022 0.600665 0.197314]]]
*********
StanDSEtaOptimize


INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


error_rate: 0.10109999999999997
log_score: -2683.3130557378936
tau = [0.0965617 0.294497  0.608942 ]
pi = [[[0.5054   0.300133 0.194467]
  [0.202153 0.60078  0.197067]
  [0.251436 0.299423 0.449141]]]


In [113]:
t_C, d = smoc.m_fit_and_compute_consensus(m, t, k, w)
d

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.fit_and_consensus
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


{'tau': array([0.608736 , 0.0968126, 0.294451 ]),
 'pi': array([[0.251518, 0.299346, 0.449137],
        [0.504781, 0.300907, 0.194312],
        [0.202022, 0.600665, 0.197314]]),
 'log_p_t_C': array([[-21.6067, -27.6235, -29.0874],
        [-21.2586, -28.6581, -26.9081],
        [-22.0125, -27.1862, -27.9742],
        ...,
        [-21.6643, -28.2208, -25.7948],
        [-23.3462, -24.7595, -29.0167],
        [-21.2586, -28.6581, -26.9081]]),
 't_C': array([[9.97008e-01, 2.43025e-03, 5.62191e-04],
        [9.95886e-01, 6.08994e-04, 3.50472e-03],
        [9.91828e-01, 5.61731e-03, 2.55448e-03],
        ...,
        [9.82806e-01, 1.39640e-03, 1.57976e-02],
        [8.02069e-01, 1.95167e-01, 2.76407e-03],
        [9.95886e-01, 6.08994e-04, 3.50472e-03]])}

In [114]:
m[:,1]=0
a = ds.m_fit_and_compute_consensus(m, t, k, 1)
a


(array([[2.52486006e-03, 5.57473873e-04, 9.96917666e-01],
        [6.29989077e-04, 3.46783611e-03, 9.95902175e-01],
        [5.80203340e-03, 2.53688186e-03, 9.91661085e-01],
        ...,
        [1.43637310e-03, 1.56576154e-02, 9.82906011e-01],
        [1.99541168e-01, 2.74859433e-03, 7.97710238e-01],
        [6.29989077e-04, 3.46783611e-03, 9.95902175e-01]]),
 {'tau': array([0.09724199, 0.29425363, 0.60850438]),
  'pi': array([[[0.50463265, 0.30035447, 0.19501288],
          [0.20209946, 0.60083436, 0.19706619],
          [0.25132606, 0.29942403, 0.44924991]]])})

In [ ]:
d = smoc.m_fit(m, t, k, w, t_C)
d

In [ ]:
d2 = smoc.m_compute_consensus(m, t, k, w, data=d)
d2

In [118]:
import math

def nc(eta):
    return (1/eta)*(1/(math.log(1-eta)-math.log(eta)))

In [119]:
nc(0.9)

-0.5056884592371319

In [120]:
nc(0.1)

4.551196133134187